In [1]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV, KFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
#file handling
from pathlib import Path
import os

In [2]:
# define the paths to all experiment data
path_group_0 = "/Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes"
path_group_1 = "/Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [3]:
FILE_PREFIX = "Enterobacter"
DATASET_OUT = "./04_UnstratifiedK/data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./04_UnstratifiedK/result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [4]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [5]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [6]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [7]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [8]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [9]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [10]:
SCORING = ['accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

In [ ]:
cnt_jobs=40

# Define number of threads/cores to use

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [11]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_group_0, path_group_1], ['d_E._aerogenes', 'e_E._cloacae'])
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae
root - INFO - Finished loading data.
root - INFO - Finished loading

# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [12]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Analyzed 120 spectra in 0.27 seconds.
Mean Score: 1513

1st Quartile: 944
Median Score: 1217
3rd Quartile: 1756

Min Score: 516
Max Score: 4098


# Preprocess the data

In [13]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

data_prep.to_csv(path_out, index=False)

# Implement Unstratified KFold CV

In [14]:
path_in = PREP_OUT

data = load_data(path_in)

X = data.drop(columns=["label", "file"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

kfold = KFold(n_splits=10)

## LDA Dimensionality Reduction
### Baseline with LDA alone

In [15]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=kfold, scoring=SCORING, return_train_score=True, verbose=3, n_jobs=4)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)
result

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.03315115, 0.008214  , 0.00634694, 0.00609112, 0.00732207,
        0.00587106, 0.00632882, 0.00549197, 0.00579309, 0.0079751 ]),
 'score_time': array([0.00128293, 0.00112891, 0.00136614, 0.00103116, 0.00129914,
        0.00103688, 0.00101185, 0.00101089, 0.00108004, 0.00110102]),
 'test_accuracy': array([0.08333333, 0.25      , 0.41666667, 0.58333333, 0.83333333,
        0.75      , 0.83333333, 0.91666667, 0.83333333, 0.66666667]),
 'train_accuracy': array([0.98148148, 0.99074074, 1.        , 0.97222222, 0.98148148,
        0.98148148, 0.9537037 , 0.98148148, 0.98148148, 0.96296296]),
 'test_f1': array([0.        , 0.        , 0.46153846, 0.73684211, 0.90909091,
        0.85714286, 0.90909091, 0.95652174, 0.90909091, 0.8       ]),
 'train_f1': array([0.98888889, 0.99441341, 1.        , 0.98089172, 0.98734177,
        0.98734177, 0.96815287, 0.98734177, 0.98734177, 0.97435897])}

### Feature Selection with PCA followed by LDA

In [16]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3, n_jobs=cnt_jobs).fit(X, y)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)



Fitting 10 folds for each of 5 candidates, totalling 50 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Inte

### Feature Selection with Nonnegative Matric Factorization followed by LDA

In [17]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3, n_jobs=cnt_jobs).fit(X, y)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/nmf_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


### Feature Selection with Feature Agglomeration followed by LDA

Fitting 10 folds for each of 5 candidates, totalling 50 fits


/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


[CV 1/10] END pca__n_components=1; accuracy: (train=0.833, test=0.000) f1: (train=0.908, test=0.000) total time=   0.3s


/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


[CV 2/10] END pca__n_components=1; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.3s


...

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


[CV 9/10] END pca__n_components=1; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.3s


/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


[CV 10/10] END pca__n_components=1; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.2s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecate

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


### Feature Selection with Feature Agglomeration followed by LDA

In [18]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)
print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 8 candidates, totalling 80 fits
[CV 1/10] END agglo__n_clusters=5; accuracy: (train=0.907, test=0.000) f1: (train=0.946, test=0.000) total time=   0.0s
[CV 2/10] END agglo__n_clusters=5; accuracy: (train=0.852, test=0.417) f1: (train=0.918, test=0.000) total time=   0.0s
[CV 3/10] END agglo__n_clusters=5; accuracy: (train=0.833, test=0.333) f1: (train=0.901, test=0.333) total time=   0.0s
[CV 4/10] END agglo__n_clusters=5; accuracy: (train=0.769, test=0.833) f1: (train=0.850, test=0.909) total time=   0.0s
[CV 5/10] END agglo__n_clusters=5; accuracy: (train=0.713, test=1.000) f1: (train=0.823, test=1.000) total time=   0.0s
[CV 6/10] END agglo__n_clusters=5; accuracy: (train=0.769, test=1.000) f1: (train=0.854, test=1.000) total time=   0.0s
[CV 7/10] END agglo__n_clusters=5; accuracy: (train=0.778, test=1.000) f1: (train=0.860, test=1.000) total time=   0.0s
[CV 8/10] END agglo__n_clusters=5; accuracy: (train=0.704, test=0.917) f1: (train=0.816, test=0.957

### Feature Selection with PeakPicker followed by LDA

In [19]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 10 folds for each of 15 candidates, totalling 150 fits
[CV 1/10] END peaks__min_dist=10; accuracy: (train=0.880, test=0.000) f1: (train=0.931, test=0.000) total time=   0.0s
[CV 2/10] END peaks__min_dist=10; accuracy: (train=0.907, test=0.167) f1: (train=0.945, test=0.000) total time=   0.0s
[CV 3/10] END peaks__min_dist=10; accuracy: (train=0.898, test=0.250) f1: (train=0.936, test=0.308) total time=   0.0s
[CV 4/10] END peaks__min_dist=10; accuracy: (train=0.824, test=0.667) f1: (train=0.881, test=0.800) total time=   0.0s
[CV 5/10] END peaks__min_dist=10; accuracy: (train=0.796, test=0.833) f1: (train=0.863, test=0.909) total time=   0.0s
[CV 6/10] END peaks__min_dist=10; accuracy: (train=0.796, test=0.917) f1: (train=0.864, test=0.957) total time=   0.0s
[CV 7/10] END peaks__min_dist=10; accuracy: (train=0.806, test=1.000) f1: (train=0.870, test=1.000) total time=   0.0s
[CV 8/10] END peaks__min_dist=10; accuracy: (train=0.778, test=1.000) f1: (train=0.854, test=1.000) tota

## Regularized Models
### Logistic Regression L1 Penalty

In [20]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END logreg__C=0.01; accuracy: (train=0.167, test=1.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 2/10] END logreg__C=0.01; accuracy: (train=0.167, test=1.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 3/10] END logreg__C=0.01; accuracy: (train=0.222, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 4/10] END logreg__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 5/10] END logreg__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 6/10] END logreg__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 7/10] END logreg__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 8/10] END logreg__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 9/10] END log

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor pre

[CV 8/10] END logreg__C=0.06309573444801933; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 9/10] END logreg__C=0.06309573444801933; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 10/10] END logreg__C=0.06309573444801933; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 1/10] END logreg__C=0.1; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 2/10] END logreg__C=0.1; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 3/10] END logreg__C=0.1; accuracy: (train=0.889, test=0.500) f1: (train=0.933, test=0.667) total time=   0.0s
[CV 4/10] END logreg__C=0.1; accuracy: (train=0.880, test=0.833) f1: (train=0.923, test=0.909) total time=   0.0s
[CV 5/10] END logreg__C=0.1; accuracy: (train=0.843, test=1.000) f1: (train=0.902, test=1.000) total time=   0.0s
[CV 6/10] END logreg__C=0.1; accuracy: 

### Logistic Regression L2 Penalty

In [21]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV 1/10] END logreg__C=1e-05; accuracy: (train=0.704, test=0.583) f1: (train=0.797, test=0.000) total time=   0.0s
[CV 2/10] END logreg__C=1e-05; accuracy: (train=0.741, test=0.417) f1: (train=0.825, test=0.000) total time=   0.0s
[CV 3/10] END logreg__C=1e-05; accuracy: (train=0.685, test=0.083) f1: (train=0.754, test=0.000) total time=   0.0s
[CV 4/10] END logreg__C=1e-05; accuracy: (train=0.722, test=0.000) f1: (train=0.783, test=0.000) total time=   0.0s
[CV 5/10] END logreg__C=1e-05; accuracy: (train=0.704, test=0.667) f1: (train=0.765, test=0.800) total time=   0.0s
[CV 6/10] END logreg__C=1e-05; accuracy: (train=0.676, test=0.750) f1: (train=0.741, test=0.857) total time=   0.0s
[CV 7/10] END logreg__C=1e-05; accuracy: (train=0.676, test=0.750) f1: (train=0.737, test=0.857) total time=   0.0s
[CV 8/10] END logreg__C=1e-05; accuracy: (train=0.676, test=0.917) f1: (train=0.745, test=0.957) total time=   0.0s
[CV 9/10]

### Linear SVM L1 Penalty

In [22]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=15000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END svm__C=0.001; accuracy: (train=0.167, test=1.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 2/10] END svm__C=0.001; accuracy: (train=0.167, test=1.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 3/10] END svm__C=0.001; accuracy: (train=0.222, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 4/10] END svm__C=0.001; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 5/10] END svm__C=0.001; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 6/10] END svm__C=0.001; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 7/10] END svm__C=0.001; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 8/10] END svm__C=0.001; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 9/10] END svm__C=0.001; accur

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor pre

[CV 9/10] END svm__C=0.00630957344480193; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 10/10] END svm__C=0.00630957344480193; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 1/10] END svm__C=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 2/10] END svm__C=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 3/10] END svm__C=0.01; accuracy: (train=0.778, test=0.500) f1: (train=0.875, test=0.667) total time=   0.0s
[CV 4/10] END svm__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 5/10] END svm__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 6/10] END svm__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 7/10] END svm__C=0.01; accuracy: (train=0.278, test=0.000) f1: (train

### Linear SVM L2 Penalty

In [23]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV 1/10] END svm__C=1e-05; accuracy: (train=0.713, test=0.583) f1: (train=0.805, test=0.000) total time=   0.0s
[CV 2/10] END svm__C=1e-05; accuracy: (train=0.750, test=0.417) f1: (train=0.832, test=0.000) total time=   0.0s
[CV 3/10] END svm__C=1e-05; accuracy: (train=0.676, test=0.083) f1: (train=0.745, test=0.000) total time=   0.0s
[CV 4/10] END svm__C=1e-05; accuracy: (train=0.722, test=0.000) f1: (train=0.783, test=0.000) total time=   0.0s
[CV 5/10] END svm__C=1e-05; accuracy: (train=0.704, test=0.667) f1: (train=0.765, test=0.800) total time=   0.0s
[CV 6/10] END svm__C=1e-05; accuracy: (train=0.676, test=0.833) f1: (train=0.741, test=0.909) total time=   0.0s
[CV 7/10] END svm__C=1e-05; accuracy: (train=0.685, test=0.750) f1: (train=0.746, test=0.857) total time=   0.0s
[CV 8/10] END svm__C=1e-05; accuracy: (train=0.676, test=0.917) f1: (train=0.745, test=0.957) total time=   0.0s
[CV 9/10] END svm__C=1e-05; accur

## Tree-based Models
### Basic Decision Tree

In [24]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.167) f1: (train=1.000, test=0.000) total time=   0.0s
[CV 2/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.417) f1: (train=1.000, test=0.000) total time=   0.0s
[CV 3/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.250) f1: (train=1.000, test=0.308) total time=   0.0s
[CV 4/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.500) f1: (train=1.000, test=0.667) total time=   0.0s
[CV 5/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.750) f1: (train=1.000, test=0.857) total time=   0.0s
[CV 6/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.667) f1: (train=1.000, test=0.800) total time=   0.0s
[CV 7/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.750) f1: (train=1.000, test=0.857) total time=   0.0s
[CV 8/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.833) f1: (train=1.000, test=0.909) total time=   0.0s
[CV 9/10] E

### Random Forest

In [25]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10] END colsample_bytree=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 2/10] END colsample_bytree=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.1s
[CV 3/10] END colsample_bytree=0.01; accuracy: (train=0.778, test=0.500) f1: (train=0.875, test=0.667) total time=   0.0s
[CV 4/10] END colsample_bytree=0.01; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 5/10] END colsample_bytree=0.01; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 6/10] END colsample_bytree=0.01; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 7/10] END colsample_bytree=0.01; accuracy: (train=0.722, test=1.000) f1: (train=0.839, test=1.000) total time=   0.0s
[CV 8/10] END colsample_bytree=0.01; accuracy: (train=0.722, test=1.000) f1: (train

### Gradient-boosted Decision Trees

In [26]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=kfold, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10] END learning_rate=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 2/10] END learning_rate=0.01; accuracy: (train=0.833, test=0.000) f1: (train=0.909, test=0.000) total time=   0.0s
[CV 3/10] END learning_rate=0.01; accuracy: (train=0.898, test=0.500) f1: (train=0.939, test=0.667) total time=   0.0s
[CV 4/10] END learning_rate=0.01; accuracy: (train=0.880, test=0.917) f1: (train=0.922, test=0.957) total time=   0.0s
[CV 5/10] END learning_rate=0.01; accuracy: (train=0.889, test=1.000) f1: (train=0.929, test=1.000) total time=   0.0s
[CV 6/10] END learning_rate=0.01; accuracy: (train=0.889, test=1.000) f1: (train=0.929, test=1.000) total time=   0.0s
[CV 7/10] END learning_rate=0.01; accuracy: (train=0.889, test=1.000) f1: (train=0.929, test=1.000) total time=   0.0s
[CV 8/10] END learning_rate=0.01; accuracy: (train=0.880, test=1.000) f1: (train=0.923, test=1.000) tota